In [1]:
import tsfel
import re
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import os

# 解析 cut_point 欄位為整數列表

In [2]:
def check_cut_point(interval):
    # 修正區間長度 <= 12 (n_coeff)，
    size = 12
    lst = [interval[0]]

    for i in range(1, len(interval)):
        if interval[i]-lst[-1] > size:
            lst.append(interval[i])

    return lst


# 解析 cut_point 欄位為整數列表
def parse_cut_point_string(s):
    if isinstance(s, str):
        s = s.strip().replace('[', '').replace(']', '')
        return [int(x) for x in re.split(r'\s+', s.strip()) if x]
    return []

# TSFEL抽取訓練集特徵

In [ ]:

# 避免抽取 spectrogram 特徵，減少欄位爆炸
cfg_file = tsfel.get_features_by_domain()
cfg_file = {k: {f: v for f, v in feats.items() if 'spectrogram' not in f.lower()} for k, feats in cfg_file.items()}

fs = 85
output_folder = "TSFEL/train"
os.makedirs(output_folder, exist_ok=True)

train_path = r"data/39_Training_Dataset/train_data"

start_time = time.time()

for mode in range(1, 11):
    try:
        del df_tsfel
    except:
        pass
    
    
    train_file = rf"new_cut_train_1/train_info_mode{mode}_updated.csv"
    df = pd.read_csv(train_file)

    df_mode = df[df['mode'] == mode]

    for i in tqdm(df_mode.index, desc=f"Process mode {mode}"):
        file = os.path.join(train_path, str(df_mode.loc[i, 'unique_id']) + ".txt")
        if not os.path.exists(file):
            print(f"❌ File not found: {file}")
            continue

        df_data = pd.read_csv(file, sep=' ', names=['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz'])
        cut_points = parse_cut_point_string(df_mode.loc[i, 'cut_point'])
        cut_points = check_cut_point(cut_points)  # ✅ 你原本有這個函數，這裡假設存在

        df_row_data = df_mode.loc[[i], ['unique_id', 'player_id', 'mode', 'gender', 
                                        'hold racket handed', 'play years', 'level']].reset_index(drop=True)

        try:
            for idx in range(len(cut_points) - 1):
                start_idx = cut_points[idx]
                end_idx = cut_points[idx + 1]
                segment = df_data.iloc[start_idx:end_idx, :]

                df_fea = tsfel.time_series_features_extractor(cfg_file, segment, fs=fs, verbose=0)
                df_fea['interval'] = end_idx - start_idx
                df_fea['cut_point_start'] = start_idx
                df_fea['cut_point_end'] = end_idx

                # 加入中繼資料
                df_fea = pd.concat([df_fea, df_row_data], axis=1)

                try:
                    df_tsfel = pd.concat([df_tsfel, df_fea], ignore_index=True)
                except:
                    df_tsfel = df_fea.copy()
        except Exception as e:
            print(f"❗ Error in mode {mode}, file: {file}")
            print(e)
    
    output_path = os.path.join(output_folder, f"df_tsfel_train_mode{mode}.csv")
    df_tsfel.to_csv(output_path, index=False)
    print(f"✅ mode{mode} 已儲存至：{output_path}")


end_time = time.time()
print("總執行時間：", round((end_time - start_time) / 60, 2), "分鐘")


Process mode 1: 100%|██████████| 42/42 [00:29<00:00,  1.41it/s]


✅ mode1 已儲存至：train/df_tsfel_train_mode1.csv


Process mode 2: 100%|██████████| 42/42 [00:30<00:00,  1.38it/s]


✅ mode2 已儲存至：train/df_tsfel_train_mode2.csv


Process mode 3: 100%|██████████| 42/42 [00:30<00:00,  1.36it/s]


...

✅ mode8 已儲存至：train/df_tsfel_train_mode8.csv


Process mode 9:  14%|█▍        | 110/785 [01:22<08:38,  1.30it/s]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(signal)
/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:554: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(signal)
Process mode 9:  15%|█▍        | 116/785 [01:26<08:35,  1.30it/s]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be 

✅ mode9 已儲存至：train/df_tsfel_train_mode9.csv


Process mode 10:  17%|█▋        | 137/829 [01:42<09:00,  1.28it/s]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(signal)
/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:554: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(signal)
Process mode 10:  17%|█▋        | 141/829 [01:46<08:58,  1.28it/s]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may b

✅ mode10 已儲存至：train/df_tsfel_train_mode10.csv
總執行時間： 31.62 分鐘


# TSFEL抽取測試集特徵

In [ ]:

# 避免抽取 spectrogram 特徵，減少欄位爆炸
cfg_file = tsfel.get_features_by_domain()
cfg_file = {k: {f: v for f, v in feats.items() if 'spectrogram' not in f.lower()} for k, feats in cfg_file.items()}

fs = 85
output_folder = "TSFEL/test"
os.makedirs(output_folder, exist_ok=True)

test_path = r"data/39_Test_Dataset/test_data"

start_time = time.time()

for mode in range(1, 11):
    try:
        del df_tsfel
    except:
        pass
    
    
    test_file = rf"new_cut_test_1/test_info_mode{mode}_updated.csv"
    df = pd.read_csv(test_file)

    df_mode = df[df['mode'] == mode]

    for i in tqdm(df_mode.index, desc=f"Process mode {mode}"):
        file = os.path.join(test_path, str(df_mode.loc[i, 'unique_id']) + ".txt")
        if not os.path.exists(file):
            print(f"❌ File not found: {file}")
            continue

        df_data = pd.read_csv(file, sep=' ', names=['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz'])
        cut_points = parse_cut_point_string(df_mode.loc[i, 'cut_point'])
        cut_points = check_cut_point(cut_points)  # ✅ 你原本有這個函數，這裡假設存在

        df_row_data = df_mode.loc[[i], ['unique_id', 'mode']].reset_index(drop=True)

        try:
            for idx in range(len(cut_points) - 1):
                start_idx = cut_points[idx]
                end_idx = cut_points[idx + 1]
                segment = df_data.iloc[start_idx:end_idx, :]


                df_fea = tsfel.time_series_features_extractor(cfg_file, segment, fs=fs, verbose=0)
                df_fea['interval'] = end_idx - start_idx
                df_fea['cut_point_start'] = start_idx
                df_fea['cut_point_end'] = end_idx

                # 加入中繼資料
                df_fea = pd.concat([df_fea, df_row_data], axis=1)

                try:
                    df_tsfel = pd.concat([df_tsfel, df_fea], ignore_index=True)
                except:
                    df_tsfel = df_fea.copy()
        except Exception as e:
            print(f"❗ Error in mode {mode}, file: {file}")
            print(e)
    
    output_path = os.path.join(output_folder, f"df_tsfel_test_mode{mode}.csv")
    df_tsfel.to_csv(output_path, index=False)
    print(f"✅ mode{mode} 已儲存至：{output_path}")


end_time = time.time()
print("總執行時間：", round((end_time - start_time) / 60, 2), "分鐘")


Process mode 1:  97%|█████████▋| 31/32 [00:21<00:00,  1.45it/s]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(signal)
/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:554: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(signal)
Process mode 1: 100%|██████████| 32/32 [00:22<00:00,  1.45it/s]


✅ mode1 已儲存至：test/df_tsfel_test_mode1.csv


Process mode 2: 100%|██████████| 31/31 [00:22<00:00,  1.40it/s]


✅ mode2 已儲存至：test/df_tsfel_test_mode2.csv


Process mode 3: 100%|██████████| 31/31 [00:22<00:00,  1.37it/s]


...

✅ mode8 已儲存至：test/df_tsfel_test_mode8.csv


Process mode 9: 100%|██████████| 476/476 [06:52<00:00,  1.15it/s]


✅ mode9 已儲存至：test/df_tsfel_test_mode9.csv


Process mode 10:  19%|█▉        | 133/705 [01:35<07:22,  1.29it/s]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(signal)
/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:554: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(signal)
Process mode 10:  62%|██████▏   | 440/705 [06:16<04:38,  1.05s/it]/home/alec/.conda/envs/tsfel/lib/python3.8/site-packages/tsfel/feature_extraction/features.py:535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may b

✅ mode10 已儲存至：test/df_tsfel_test_mode10.csv
總執行時間： 21.28 分鐘
